In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [3]:
from june import World 
from june.demography.geography import Geography
from june.demography import Demography
from june.interaction import ContactAveraging
from june.infection import Infection, HealthIndexGenerator
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import generate_leisure_for_config, Cinemas, Pubs, Groceries
from june.simulator import Simulator
from june.infection_seed import InfectionSeed
from june.policy import Policy, Policies
from june import paths
from june.hdf5_savers import load_geography_from_hdf5
from june.logger.read_logger import ReadLogger
from june.infection.infection import InfectionSelector
from june.world import generate_world_from_hdf5, generate_world_from_geography

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/configs


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [4]:
CONFIG_PATH = "../configs/config_example.yaml"

In [5]:
%%time 

geography = Geography.from_file(
        {
            "super_area": [
                "E02003282",
                "E02001720",
                "E02002560",
                "E02002559",
                "E02004314",
            ]
        }
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True, include_commute=True)

2020-07-21 18:04:37,800 - june.demography.geography - INFO - There are 129 areas and 5 super_areas in the world.
2020-07-21 18:04:37,808 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-07-21 18:04:37,873 - june.groups.school - INFO - There are 19 schools in this geography.
2020-07-21 18:04:37,923 - june.groups.school - INFO - No school for the age 0 in this world.
2020-07-21 18:04:37,928 - june.groups.school - INFO - No school for the age 1 in this world.
2020-07-21 18:04:37,934 - june.groups.school - INFO - No school for the age 2 in this world.
2020-07-21 18:04:38,304 - june.groups.carehome - INFO - There are 9 care_homes in this geography.
2020-07-21 18:04:47,234 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-07-21 18:04:53,427 - june.distributors.worker_distributor - INFO - There are 0 who had to be told to stay real
CPU times: user 41 s, sys: 2 s, total: 43 s
Wall time: 44.5 s


## Commute, travel and leisure

In [6]:
world.pubs = Pubs.for_geography(geography)
world.cinemas = Cinemas.for_geography(geography)
world.groceries = Groceries.for_geography(geography)
leisure = generate_leisure_for_config(world, config_filename=CONFIG_PATH)
leisure.distribute_social_venues_to_households(world.households) # this assigns possible social venues to people.

We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [7]:
world.cemeteries = Cemeteries()

In [8]:
len(world.people)

46784

### If it took a long time to run the previous commands, it might be a good idea to save the world to reuse it later.

In [9]:
world.to_hdf5("world.hdf5")

If we would like to load the world we saved, we just do

In [10]:
world = generate_world_from_hdf5("world.hdf5")

loading population from hdf5 .
loading companies from hdf5 .
loading households from hdf5 .

  0%|          | 0/46784 [00:00<?, ?it/s]


restoring world...


100%|██████████| 46784/46784 [00:01<00:00, 41528.16it/s]


In [11]:
world.pubs

In [12]:
# and regenerate leisure in case we load it externally
leisure = generate_leisure_for_config(world, CONFIG_PATH)

you have now a beautiful pre-pandemic world. 

# Adding the infection

Now, you can play around with different models of infections. The building blocks for an infection are: Transmission and Symptoms. For now, only SymptomsConstant and TransmissionConstant are in a state to be used.

In [13]:
health_index_generator = HealthIndexGenerator.from_file(asymptomatic_ratio=0.2)
selector = InfectionSelector.from_file(health_index_generator=health_index_generator)

In [14]:
selector.health_index_generator.asymptomatic_ratio

0.2

# Adding the interaction

In [15]:
interaction = ContactAveraging.from_file(selector=selector)

Beta are the intensities of the interaction taking place at the different groups

In [16]:
interaction.beta

{'box': 1,
 'pub': 0.1,
 'grocery': 0.1,
 'cinema': 0.1,
 'commute_unit': 1.0,
 'commute_city_unit': 1.0,
 'hospital': 1.0,
 'care_home': 1.0,
 'company': 0.1,
 'school': 0.1,
 'household': 0.2,
 'university': 0.01}

to modify these, simply do

In [17]:
for key in interaction.beta.keys():
    interaction.beta[key] *= 2

In [18]:
interaction.beta

{'box': 2,
 'pub': 0.2,
 'grocery': 0.2,
 'cinema': 0.2,
 'commute_unit': 2.0,
 'commute_city_unit': 2.0,
 'hospital': 2.0,
 'care_home': 2.0,
 'company': 0.2,
 'school': 0.2,
 'household': 0.4,
 'university': 0.02}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [19]:
interaction.alpha_physical

2.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [20]:
infection_seed = InfectionSeed(
    world.super_areas, selector,
)

In [21]:
n_cases = 50
infection_seed.unleash_virus(n_cases) # play around with the initial number of cases

# Set policies

In [22]:
policies = Policies.from_file()

# Run the simulation

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [23]:
simulator = Simulator.from_file(
     world, interaction, 
    config_filename = CONFIG_PATH,
    leisure = leisure,
    policies = policies
)

In [24]:
%%time
simulator.run()

2020-07-21 18:05:47,653 - june.simulator - INFO - Starting group_dynamics for 250 days at day 0
2020-07-21 18:05:47,655 - june.simulator - INFO - starting the loop ..., at 0 days, to run for 250 days
2020-07-21 18:05:49,164 - june.simulator - INFO - Date = 2020-03-01 04:00:00, number of deaths =  0, number of infected = 50
2020-07-21 18:05:50,166 - june.simulator - INFO - Date = 2020-03-01 08:00:00, number of deaths =  0, number of infected = 50
2020-07-21 18:05:50,768 - june.simulator - INFO - Date = 2020-03-01 12:00:00, number of deaths =  0, number of infected = 50
2020-07-21 18:05:51,450 - june.simulator - INFO - Date = 2020-03-02 00:00:00, number of deaths =  0, number of infected = 50
2020-07-21 18:05:52,101 - june.simulator - INFO - Date = 2020-03-02 01:00:00, number of deaths =  0, number of infected = 51
2020-07-21 18:05:52,570 - june.simulator - INFO - Date = 2020-03-02 09:00:00, number of deaths =  0, number of infected = 52
2020-07-21 18:05:53,584 - june.simulator - INFO - 

2020-07-21 18:06:37,789 - june.simulator - INFO - Date = 2020-03-14 12:00:00, number of deaths =  0, number of infected = 1073
2020-07-21 18:06:39,072 - june.simulator - INFO - Date = 2020-03-15 00:00:00, number of deaths =  0, number of infected = 1143
2020-07-21 18:06:40,131 - june.simulator - INFO - Date = 2020-03-15 04:00:00, number of deaths =  0, number of infected = 1196
2020-07-21 18:06:41,117 - june.simulator - INFO - Date = 2020-03-15 08:00:00, number of deaths =  0, number of infected = 1243
2020-07-21 18:06:41,844 - june.simulator - INFO - Date = 2020-03-15 12:00:00, number of deaths =  0, number of infected = 1294
2020-07-21 18:06:43,038 - june.simulator - INFO - Date = 2020-03-16 00:00:00, number of deaths =  0, number of infected = 1392
2020-07-21 18:06:44,380 - june.simulator - INFO - Date = 2020-03-16 01:00:00, number of deaths =  0, number of infected = 1451
2020-07-21 18:06:45,418 - june.simulator - INFO - Date = 2020-03-16 09:00:00, number of deaths =  0, number of 

2020-07-21 18:08:05,792 - june.simulator - INFO - Date = 2020-03-28 08:00:00, number of deaths =  4, number of infected = 6148
2020-07-21 18:08:06,904 - june.simulator - INFO - Date = 2020-03-28 12:00:00, number of deaths =  4, number of infected = 6199
2020-07-21 18:08:08,784 - june.simulator - INFO - Date = 2020-03-29 00:00:00, number of deaths =  5, number of infected = 6370
2020-07-21 18:08:10,086 - june.simulator - INFO - Date = 2020-03-29 04:00:00, number of deaths =  5, number of infected = 6434
2020-07-21 18:08:11,330 - june.simulator - INFO - Date = 2020-03-29 08:00:00, number of deaths =  5, number of infected = 6479
2020-07-21 18:08:12,624 - june.simulator - INFO - Date = 2020-03-29 12:00:00, number of deaths =  5, number of infected = 6539
2020-07-21 18:08:14,198 - june.simulator - INFO - Date = 2020-03-30 00:00:00, number of deaths =  5, number of infected = 6654
2020-07-21 18:08:15,589 - june.simulator - INFO - Date = 2020-03-30 01:00:00, number of deaths =  5, number of 

2020-07-21 18:09:32,209 - june.simulator - INFO - Date = 2020-04-11 04:00:00, number of deaths =  27, number of infected = 8376
2020-07-21 18:09:33,448 - june.simulator - INFO - Date = 2020-04-11 08:00:00, number of deaths =  27, number of infected = 8378
2020-07-21 18:09:34,520 - june.simulator - INFO - Date = 2020-04-11 12:00:00, number of deaths =  27, number of infected = 8381
2020-07-21 18:09:35,916 - june.simulator - INFO - Date = 2020-04-12 00:00:00, number of deaths =  28, number of infected = 8306
2020-07-21 18:09:37,266 - june.simulator - INFO - Date = 2020-04-12 04:00:00, number of deaths =  28, number of infected = 8293
2020-07-21 18:09:38,640 - june.simulator - INFO - Date = 2020-04-12 08:00:00, number of deaths =  28, number of infected = 8282
2020-07-21 18:09:39,632 - june.simulator - INFO - Date = 2020-04-12 12:00:00, number of deaths =  29, number of infected = 8252
2020-07-21 18:09:41,047 - june.simulator - INFO - Date = 2020-04-13 00:00:00, number of deaths =  29, nu

2020-07-21 18:10:58,542 - june.simulator - INFO - Date = 2020-04-25 00:00:00, number of deaths =  44, number of infected = 5532
2020-07-21 18:10:59,841 - june.simulator - INFO - Date = 2020-04-25 04:00:00, number of deaths =  44, number of infected = 5484
2020-07-21 18:11:01,094 - june.simulator - INFO - Date = 2020-04-25 08:00:00, number of deaths =  44, number of infected = 5437
2020-07-21 18:11:02,044 - june.simulator - INFO - Date = 2020-04-25 12:00:00, number of deaths =  44, number of infected = 5396
2020-07-21 18:11:03,208 - june.simulator - INFO - Date = 2020-04-26 00:00:00, number of deaths =  47, number of infected = 5258
2020-07-21 18:11:04,261 - june.simulator - INFO - Date = 2020-04-26 04:00:00, number of deaths =  47, number of infected = 5215
2020-07-21 18:11:05,366 - june.simulator - INFO - Date = 2020-04-26 08:00:00, number of deaths =  47, number of infected = 5191
2020-07-21 18:11:06,418 - june.simulator - INFO - Date = 2020-04-26 12:00:00, number of deaths =  47, nu

...

2020-07-21 18:21:04,967 - june.simulator - INFO - Date = 2020-09-10 00:00:00, number of deaths =  76, number of infected = 3747
2020-07-21 18:21:06,037 - june.simulator - INFO - Date = 2020-09-10 01:00:00, number of deaths =  76, number of infected = 3749
2020-07-21 18:21:06,910 - june.simulator - INFO - Date = 2020-09-10 09:00:00, number of deaths =  76, number of infected = 3791
2020-07-21 18:21:07,932 - june.simulator - INFO - Date = 2020-09-10 10:00:00, number of deaths =  76, number of infected = 3784
2020-07-21 18:21:08,595 - june.simulator - INFO - Date = 2020-09-10 13:00:00, number of deaths =  76, number of infected = 3797
2020-07-21 18:21:09,467 - june.simulator - INFO - Date = 2020-09-11 00:00:00, number of deaths =  76, number of infected = 3797
2020-07-21 18:21:10,487 - june.simulator - INFO - Date = 2020-09-11 01:00:00, number of deaths =  76, number of infected = 3800
2020-07-21 18:21:11,424 - june.simulator - INFO - Date = 2020-09-11 09:00:00, number of deaths =  78, nu

2020-07-21 18:22:03,705 - june.simulator - INFO - Date = 2020-09-23 13:00:00, number of deaths =  87, number of infected = 3951
2020-07-21 18:22:04,602 - june.simulator - INFO - Date = 2020-09-24 00:00:00, number of deaths =  88, number of infected = 3937
2020-07-21 18:22:05,502 - june.simulator - INFO - Date = 2020-09-24 01:00:00, number of deaths =  88, number of infected = 3932
2020-07-21 18:22:06,383 - june.simulator - INFO - Date = 2020-09-24 09:00:00, number of deaths =  89, number of infected = 3931
2020-07-21 18:22:07,204 - june.simulator - INFO - Date = 2020-09-24 10:00:00, number of deaths =  89, number of infected = 3930
2020-07-21 18:22:07,779 - june.simulator - INFO - Date = 2020-09-24 13:00:00, number of deaths =  89, number of infected = 3929
2020-07-21 18:22:08,507 - june.simulator - INFO - Date = 2020-09-25 00:00:00, number of deaths =  90, number of infected = 3905
2020-07-21 18:22:09,420 - june.simulator - INFO - Date = 2020-09-25 01:00:00, number of deaths =  90, nu

2020-07-21 18:22:53,100 - june.simulator - INFO - Date = 2020-10-07 10:00:00, number of deaths =  98, number of infected = 3390
2020-07-21 18:22:53,580 - june.simulator - INFO - Date = 2020-10-07 13:00:00, number of deaths =  98, number of infected = 3384
2020-07-21 18:22:54,226 - june.simulator - INFO - Date = 2020-10-08 00:00:00, number of deaths =  99, number of infected = 3354
2020-07-21 18:22:54,750 - june.simulator - INFO - Date = 2020-10-08 01:00:00, number of deaths =  99, number of infected = 3347
2020-07-21 18:22:55,386 - june.simulator - INFO - Date = 2020-10-08 09:00:00, number of deaths =  99, number of infected = 3334
2020-07-21 18:22:56,088 - june.simulator - INFO - Date = 2020-10-08 10:00:00, number of deaths =  99, number of infected = 3331
2020-07-21 18:22:56,681 - june.simulator - INFO - Date = 2020-10-08 13:00:00, number of deaths =  99, number of infected = 3330
2020-07-21 18:22:57,331 - june.simulator - INFO - Date = 2020-10-09 00:00:00, number of deaths =  100, n

2020-07-21 18:23:33,322 - june.simulator - INFO - Date = 2020-10-21 01:00:00, number of deaths =  106, number of infected = 2304
2020-07-21 18:23:33,793 - june.simulator - INFO - Date = 2020-10-21 09:00:00, number of deaths =  106, number of infected = 2281
2020-07-21 18:23:34,498 - june.simulator - INFO - Date = 2020-10-21 10:00:00, number of deaths =  106, number of infected = 2278
2020-07-21 18:23:34,980 - june.simulator - INFO - Date = 2020-10-21 13:00:00, number of deaths =  106, number of infected = 2271
2020-07-21 18:23:35,501 - june.simulator - INFO - Date = 2020-10-22 00:00:00, number of deaths =  107, number of infected = 2206
2020-07-21 18:23:36,021 - june.simulator - INFO - Date = 2020-10-22 01:00:00, number of deaths =  107, number of infected = 2204
2020-07-21 18:23:36,472 - june.simulator - INFO - Date = 2020-10-22 09:00:00, number of deaths =  107, number of infected = 2179
2020-07-21 18:23:37,152 - june.simulator - INFO - Date = 2020-10-22 10:00:00, number of deaths = 

2020-07-21 18:24:11,121 - june.simulator - INFO - Date = 2020-11-03 13:00:00, number of deaths =  113, number of infected = 1279
2020-07-21 18:24:11,628 - june.simulator - INFO - Date = 2020-11-04 00:00:00, number of deaths =  113, number of infected = 1256
2020-07-21 18:24:12,138 - june.simulator - INFO - Date = 2020-11-04 01:00:00, number of deaths =  113, number of infected = 1252
2020-07-21 18:24:12,566 - june.simulator - INFO - Date = 2020-11-04 09:00:00, number of deaths =  113, number of infected = 1243
2020-07-21 18:24:13,340 - june.simulator - INFO - Date = 2020-11-04 10:00:00, number of deaths =  113, number of infected = 1245
2020-07-21 18:24:13,767 - june.simulator - INFO - Date = 2020-11-04 13:00:00, number of deaths =  113, number of infected = 1240
2020-07-21 18:24:14,316 - june.simulator - INFO - Date = 2020-11-05 00:00:00, number of deaths =  113, number of infected = 1240
2020-07-21 18:24:14,813 - june.simulator - INFO - Date = 2020-11-05 01:00:00, number of deaths = 

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [25]:
import pandas as pd

In [26]:
read = ReadLogger()

## Hospital data and how it changed over time

In [27]:
hospitals_df = read.load_hospital_capacity()

In [28]:
hospitals_df.head(3)

,time_stamp,id,n_patients,n_patients_icu
0,2020-03-01T04:00:00.000000,0,0,0
1,2020-03-01T08:00:00.000000,0,0,0
2,2020-03-01T12:00:00.000000,0,0,0


In [29]:
hospitals_characteristics_df = read.load_hospital_characteristics()

In [30]:
hospitals_characteristics_df

,longitude,latitude,n_beds,n_icu_beds,trust_code
0,0.688477,51.5625,2115,296,RAJ


## where did infections happen?

In [ ]:
loc_df = read.get_locations_infections()

In [ ]:
locations_per_day = read.locations_df.groupby(pd.Grouper(freq='D')).sum()

In [ ]:
all_infection_places = set(locations_per_day.location.sum())

In [ ]:
def n_infections(row, infection_place):
    return sum([row.counts[i] for i, x in enumerate(row.location) if x == infection_place])


In [ ]:
for infection_place in all_infection_places:
    locations_per_day[str(infection_place)] = locations_per_day.apply(
        lambda x: n_infections(x, infection_place),
        axis=1
    )

In [ ]:
locations_per_day = locations_per_day.drop(columns=['location',
                                                    'counts'])

In [ ]:
locations_per_day = locations_per_day.div(
    locations_per_day.sum(axis=1), axis=0
)


In [ ]:
locations_per_day.plot.area( alpha=0.5)
plt.legend(bbox_to_anchor=(1,1))
plt.ylabel('Percent of infections')

In [ ]:
from matplotlib import cm
cmap = cm.get_cmap('Spectral') # Colour map (there are many others)

locations_per_day.plot(figsize=(30,8), logy=True, ylim=(1e-3, 1), cmap=cmap)

In [ ]:
import matplotlib.ticker as mtick

ax = loc_df['percentage_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')
#plt.yscale('log')

## rate of infection

In [ ]:
r_df = read.get_r()

In [ ]:
from june.policy import SocialDistancing
r_df.plot()
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.axhline(y=1, linestyle='dashed', color='gray')
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('R')

## World infection curves, and by super area

In [ ]:
world_df = read.world_summary()

In [ ]:
#import matplotlib.pyplot as plt
 
#plt.style.use(['science','notebook'])
#plt.style.reload_library()

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
fig, ax = plt.subplots()
world_df['new_infections'].resample('D').sum().plot(ax=ax)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.axvline('2020-03-23',
           linestyle='dashed',
           color='indianred')
ax.set_ylabel('# new infections')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots()
(world_df['hospital_admissions']+world_df['intensive_care_admissions']).resample('D').sum().plot(ax=ax)
colors = ["yellow", "orange"]
for policy in policies.polici-es:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.axvline('2020-04-01',
           linestyle='dashed',
           color='indianred')
ax.set_ylabel('# hospital admissions')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots()
world_df['infected'].plot(ax=ax)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
ax.set_ylabel('# Infected')
ax.set_xlabel('Date')

In [ ]:
world_df.plot()
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


In [ ]:
area_df = read.super_area_summary()

In [ ]:
area_df[area_df.super_area == "E02003282"].plot()
plt.legend(bbox_to_anchor=(1,1))



## World infection curves per age group

In [ ]:
ages_df = read.age_summary([0,10,20,30,40,
                  50,60,70,80,90,100])

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['infected'].plot(label=name)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['dead'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['susceptible'].plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['hospital_admissions'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['intensive_care_admissions'].cumsum().plot(label=name)
plt.legend()

## Draw some of the symptoms trajectories

In [ ]:
random_trajectories = read.draw_symptom_trajectories(window_length=600,
                                        n_people=10)

In [ ]:
from june.infection import SymptomTag

In [ ]:
symptoms_values = [tag.value for tag in SymptomTag]
symptoms_names = [tag.name for tag in SymptomTag]


In [ ]:
for df_person in random_trajectories:
    df_person['symptoms'].plot()
plt.ylabel('Symptoms Trajectory')
_ = plt.yticks(symptoms_values, symptoms_names)
plt.xlabel('Date')

In [ ]:
for df_person in random_trajectories:
    df_person['n_secondary_infections'].plot()
plt.ylabel('Number of secondary infections')

plt.xlabel('Date')